(sec:regex)=
# 正規表現とスクレイピング

本節では、正規表現と、それを用いたHTMLのスクレイピングについて学ぶ。

## 正規表現の基本

In [1]:
import re

In [4]:
text = "Hello, world!"
pat = re.compile("Hello")
match = pat.search(text)
if match is not None:
    print(f"Matched text is \"{match.group(0):s}\"")
else:
    print("Pattern not matched.")

Matched text is "Hello"


正規表現は大文字と小文字を区別するので、`re.compile("hello")`のように`pat`を定義すると、マッチしなくなる。大文字小文字を区別せずに文字列のマッチを検索する方法もあるので、後述する。

### 練習問題

- 文字列の中に `"world"`という文字列が含まれるかどうかを調べるように上記の例を改良せよ。

In [8]:
## 答え
pat2 = re.compile("world")
match2 = pat2.search(text)
if match2 is not None:
    print("\"text\" includes \"world\"")
else:
    print("\"text\" does not include \"world\"")

"text" includes "world"


### マッチ・グループ

上記の例では、マッチした文字列を取り出すために `match.group(0)`のように`group`メソッドに`0`を指定した。この`group`が指すものはマッチ・グループで、`0`はマッチした文章全体を示す。

## urllib

Pythonでウェブページのデータをサーバーから取得するための標準ライブラリに`urllib`がある。

In [17]:
import urllib

try:
    req = urllib.request.urlopen("https://google.co.jp/")
except urllib.error.HTTPError as e:
    print(e)

if req is not None:
    body = req.read()
    print(f"{len(body):d} bytes received!")
else:
    print("HTTP request failed!")

18524 bytes received!


In [20]:
body = body.decode("shift-jis")

In [23]:
pat = re.compile("href=\"(.*?)\"")
match = pat.search(body)
if match is not None:
    print(match.group(1))

https://www.google.co.jp/imghp?hl=ja&tab=wi


#### 練習問題
- Googleのトップページ (https://www.google.co.jp/) から、Googleのロゴ画像のURLを取り出しなさい。